In [1]:
import optuna
import tensorflow as tf


In [3]:
import tensorflow as tf
from tensorflow.keras import layers
import optuna


# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize pixel values between 0 and 1
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Add a channel dimension (for grayscale images)
x_train = tf.expand_dims(x_train, -1)
x_test = tf.expand_dims(x_test, -1)

# Define the objective function for Optuna
def objective(trial):
    

    # Define the model architecture
    model = tf.keras.Sequential()
    
    # Add the convolutional layers
    model.add(layers.Conv2D(
        trial.suggest_int("conv1_filters", 16, 64, log=True),
        (3, 3),
        activation="relu",
        input_shape=(28, 28, 1)
    ))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(
        trial.suggest_int("conv2_filters", 32, 128, log=True),
        (3, 3),
        activation="relu"
    ))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())

    # Add the dense layers
    model.add(layers.Dense(
        trial.suggest_int("dense_units", 32, 128, log=True),
        activation="relu"
    ))

    model.add(layers.Dense(10, activation="softmax"))

    # Compile the model
    model.compile(
        optimizer="adam",
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"]
    )

    # Train the model
    model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test), verbose=1)

    # Evaluate the model
    _, accuracy = model.evaluate(x_test, y_test)

    # Return the negative accuracy (as Optuna tries to minimize the objective)
    return -accuracy

# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

# Get the best hyperparameters
best_params = study.best_params
print("Best hyperparameters:", best_params)

# Train the model with the best hyperparameters
best_model = tf.keras.Sequential()
best_model.add(layers.Conv2D(
    best_params["conv1_filters"],
    (3, 3),
    activation="relu",
    input_shape=(28, 28, 1)
))
best_model.add(layers.MaxPooling2D((2, 2)))
best_model.add(layers.Conv2D(
    best_params["conv2_filters"],
    (3, 3),
    activation="relu"
))
best_model.add(layers.MaxPooling2D((2, 2)))
best_model.add(layers.Flatten())
best_model.add(layers.Dense(
    best_params["dense_units"],
    activation="relu"
))
best_model.add(layers.Dense(10, activation="softmax"))

best_model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

best_model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

# Evaluate
_, accuracy = model.evaluate(x_test, y_test)

[I 2023-05-08 19:51:09,876] A new study created in memory with name: no-name-3d3f8d0f-d885-4c73-abe3-15ab72bd65a3


Epoch 1/5
1875/1875 [==============================] - 23s 12ms/step - loss: 0.1574 - accuracy: 0.9526 - val_loss: 0.0609 - val_accuracy: 0.9813
Epoch 2/5
1875/1875 [==============================] - 24s 13ms/step - loss: 0.0506 - accuracy: 0.9841 - val_loss: 0.0338 - val_accuracy: 0.9886
Epoch 3/5
1875/1875 [==============================] - 26s 14ms/step - loss: 0.0366 - accuracy: 0.9886 - val_loss: 0.0370 - val_accuracy: 0.9877
Epoch 4/5
1875/1875 [==============================] - 22s 12ms/step - loss: 0.0288 - accuracy: 0.9911 - val_loss: 0.0294 - val_accuracy: 0.9907
Epoch 5/5
313/313 [==============================] - 1s 4ms/step - loss: 0.0299 - accuracy: 0.9896


[I 2023-05-08 19:53:08,787] Trial 0 finished with value: -0.9896000027656555 and parameters: {'conv1_filters': 25, 'conv2_filters': 51, 'dense_units': 39}. Best is trial 0 with value: -0.9896000027656555.


Epoch 1/5
1875/1875 [==============================] - 43s 23ms/step - loss: 0.1278 - accuracy: 0.9617 - val_loss: 0.0513 - val_accuracy: 0.9831
Epoch 2/5
1875/1875 [==============================] - 42s 22ms/step - loss: 0.0404 - accuracy: 0.9874 - val_loss: 0.0327 - val_accuracy: 0.9882
Epoch 3/5
1875/1875 [==============================] - 42s 22ms/step - loss: 0.0279 - accuracy: 0.9913 - val_loss: 0.0308 - val_accuracy: 0.9908
Epoch 4/5
1875/1875 [==============================] - 43s 23ms/step - loss: 0.0200 - accuracy: 0.9934 - val_loss: 0.0327 - val_accuracy: 0.9891
Epoch 5/5
313/313 [==============================] - 2s 7ms/step - loss: 0.0318 - accuracy: 0.9908


[I 2023-05-08 19:56:42,034] Trial 1 finished with value: -0.9908000230789185 and parameters: {'conv1_filters': 58, 'conv2_filters': 57, 'dense_units': 123}. Best is trial 0 with value: -0.9896000027656555.


...

Epoch 1/5
1875/1875 [==============================] - 34s 18ms/step - loss: 0.1478 - accuracy: 0.9548 - val_loss: 0.0655 - val_accuracy: 0.9789
Epoch 2/5
1875/1875 [==============================] - 33s 18ms/step - loss: 0.0497 - accuracy: 0.9847 - val_loss: 0.0403 - val_accuracy: 0.9877
Epoch 3/5
1875/1875 [==============================] - 34s 18ms/step - loss: 0.0341 - accuracy: 0.9894 - val_loss: 0.0335 - val_accuracy: 0.9899
Epoch 4/5
1875/1875 [==============================] - 33s 18ms/step - loss: 0.0256 - accuracy: 0.9921 - val_loss: 0.0315 - val_accuracy: 0.9897
Epoch 5/5
313/313 [==============================] - 2s 6ms/step - loss: 0.0304 - accuracy: 0.9897


[I 2023-05-08 20:10:22,753] Trial 6 finished with value: -0.9897000193595886 and parameters: {'conv1_filters': 26, 'conv2_filters': 126, 'dense_units': 40}. Best is trial 2 with value: -0.987500011920929.


Epoch 1/5
1875/1875 [==============================] - 40s 21ms/step - loss: 0.1342 - accuracy: 0.9588 - val_loss: 0.0437 - val_accuracy: 0.9859
Epoch 2/5
1875/1875 [==============================] - 40s 21ms/step - loss: 0.0443 - accuracy: 0.9862 - val_loss: 0.0368 - val_accuracy: 0.9880
Epoch 3/5
 683/1875 [=========>....................] - ETA: 24s - loss: 0.0326 - accuracy: 0.9904

[W 2023-05-08 20:11:56,824] Trial 7 failed with parameters: {'conv1_filters': 58, 'conv2_filters': 45, 'dense_units': 127} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\alexw\AppData\Local\Programs\Python\Python39\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\alexw\AppData\Local\Temp\ipykernel_9000\3358571551.py", line 58, in objective
    model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test), verbose=1)
  File "C:\Users\alexw\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\utils\traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
  File "C:\Users\alexw\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1564, in fit
    tmp_logs = self.train_function(iterator)
  File "C:\Users\alexw\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\uti

KeyboardInterrupt: 

In [39]:
import importlib.util

# Specify the absolute path to the CustomCNN.py file
custom_cnn_path = 'C:/Users/alexw/Documents/Git/AI-WORK/Advanced-Ai/CNN/CustomCNN.py'

# Load the CustomCNN module
spec = importlib.util.spec_from_file_location('CustomCNN', custom_cnn_path)
CustomCNN = importlib.util.module_from_spec(spec)
spec.loader.exec_module(CustomCNN)


a = CustomCNN.CNN_Layer(3, 2, tf.keras.activations.gelu, 3, 3, "AveragePooling1D")

In [ ]:
def Optuna_Listelements(name_layer,liste,key,trial):
    if type(liste[0])==int:
        return trial.suggest_loguniform(f"{name_layer}+{key}", liste[0], liste[1]) 
    elif type(liste[0])==str:
        return trial.suggest_categorical(f"{name_layer}+{key}", liste)

In [ ]:
for k in dictionnary:
    dictionnary[k]

In [40]:
def Optimizing_Layer(trial,name_layer,layer):
    filters = trial.suggest_loguniform(f{name_layer}, 8, 256) 


{'hyperparameter_filters': ListWrapper([8, 256, 8]),
 'hyperparameter_kernel_size': ListWrapper([1, 10, 1]),
 'hyperparameter_pool_size': ListWrapper([1, 10, 1]),
 'hyperparameter_strides': ListWrapper([1, 10, 1]),
 'hyperparameter_pooling_layer_name': ListWrapper(['MetaPoolingLayer', 'AveragePooling2D', 'MaxPooling2D', 'AveragePooling1D', 'MaxPooling1D'])}

In [42]:
liste =a.get_hyperparameter()["hyperparameter_filters"]